# Introduction

If someone has an idea for a new classifier, it should compare the performance of its new classifier with the performance of standard classifiers on some standard datasets.

But what are the performances of these standard classifiers on these standard datasets?

This notebook tries to answer this question.

# TensorFlow Version check

In [6]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.0.0


# Set experiment parameters

In [2]:
PARAM_NR_TEST_IMAGES = 10000
PARAM_NR_TRAIN_EPOCHS = 300

# MNIST

## Preparing the dataset

In [ ]:
import numpy as np

def prepare_data():
    
    # 1. load MNIST dataset
    mnist_dataset = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist_dataset.load_data()
    
    # 2. map image pixel values from [0,255] to [0,1]
    x_train = x_train.astype(float)
    x_test  = x_test.astype(float)
    
    x_train = x_train * (1.0/255.0)
    x_test  = x_test  * (1.0/255.0)
    
    # 3. images are 2D, prepare input as 1D input for a MLP
    x_train = x_train.reshape(-1,28*28)
    x_test  = x_test.reshape(-1,28*28)
    
    # 4. map train data target labels to one-hot encoded vectors
    y_train_onehot = np.zeros((y_train.size, y_train.max()+1))
    y_train_onehot[np.arange(y_train.size),y_train] = 1
    
    # 5. map test data target labels to one-hot encoded vectors
    y_test_onehot = np.zeros((y_test.size, y_train.max()+1))
    y_test_onehot[np.arange(y_test.size),y_test] = 1
    
    return x_train, y_train_onehot, \
           x_test,  y_test_onehot

# get MNIST data now
x_train, y_train, x_test, y_test = prepare_data()

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train[:5]

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
from matplotlib import pyplot as plt

for i in range(0,5):
    plt.imshow( x_train[i,:].reshape(28,28), cmap="gray" )
    plt.colorbar()
    plt.show()

## Model test function

In [3]:
def model_test(your_model, x_test, y_test, how_many_to_test=None, model_type="MLP"):
    
    nr_test_samples = x_test.shape[0]
    
    if how_many_to_test is None:
        how_many_to_test = nr_test_samples
   
    correct = 0
    for sample_id in range(0, how_many_to_test):
        
        if sample_id % 100 == 0:
            print("tested so far {0} of {1} images".format(sample_id, how_many_to_test))

        # get next test sample
        input_vec   = x_test[sample_id]
        teacher_vec = y_test[sample_id]
        
        # compute prediction vector
                
        if model_type == "MLP":
            
            # note: the input to a MLP is a batch of 1D inputs, so 2D!
            pred_vec = your_model.predict( input_vec.reshape(1, -1) )
            
            
        elif model_type == "CNN":
            
            # note: the input to a CNN is a batch of 3D inputs, so 3D!
            # 1 test image
            # 28 pixel wide
            # 28 pixel high
            # 1 color channel (gray-scale image)
            # --> shape is (1,28,28,1)
            pred_vec = your_model.predict( input_vec.reshape(1, 28,28,1) )
        
        # compute predicted label
        pred_label = np.argmax( pred_vec )
        
        # get actual label
        gt_label = np.argmax( teacher_vec )
        
        #print("pred: {0} vs. gt: {1}".format(pred_label, gt_label))
        
        # was the predicted label correct?
        if pred_label == gt_label:
            correct += 1
            
    acc = correct/how_many_to_test
    return acc

## Perceptron

In [ ]:
from tensorflow import keras

tf.keras.backend.clear_session()

nr_inputs = 28*28

model = keras.Sequential()
model.add(keras.layers.Dense(10,
                             activation="linear",
                             input_shape=(nr_inputs,)))
model.compile(optimizer='sgd',               
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

In [ ]:
history = model.fit(x_train, y_train, epochs=PARAM_NR_TRAIN_EPOCHS, validation_split=0.1)

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

## MLP

In [ ]:
tf.keras.backend.clear_session()

nr_inputs = 28*28

model = keras.Sequential()
model.add(keras.layers.Dense(200,
                             activation="relu",
                             input_shape=(nr_inputs,)))
model.add(keras.layers.Dense(50,
                             activation="relu"))
model.add(keras.layers.Dense(10,
                             activation="linear"))
model.compile(optimizer='sgd',               
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

In [ ]:
history = model.fit(x_train, y_train, epochs=PARAM_NR_TRAIN_EPOCHS, validation_split=0.1)

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

## CNN

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras import losses

K.clear_session()

model = keras.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu', name="FC"))

model.add(layers.Dense(10, activation='linear', name="output"))

model.compile(loss=losses.mean_squared_error, optimizer='sgd')

model.summary()

In [ ]:
model_test(model,
           x_test,
           y_test,
           how_many_to_test=500,
           model_type="CNN"
          )

In [ ]:
history = model.fit(x_train.reshape(60000, 28,28,1),
                    y_train,
                    epochs=PARAM_NR_TRAIN_EPOCHS/2,
                    validation_split=0.1)

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

# Fashion MNIST

Why Fashion-MNIST?

See https://github.com/zalandoresearch/fashion-mnist:

Quote from the Fashion-MNIST GitHub website:

    To Serious Machine Learning Researchers

    Seriously, we are talking about replacing MNIST. Here are some good reasons:

    MNIST is too easy. Convolutional nets can achieve 99.7% on MNIST. Classic machine learning algorithms can also achieve 97% easily. Check out our side-by-side benchmark for Fashion-MNIST vs. MNIST, and read "Most pairs of MNIST digits can be distinguished pretty well by just one pixel."

    MNIST is overused. In this April 2017 Twitter thread, Google Brain research scientist and deep learning expert Ian Goodfellow calls for people to move away from MNIST.

    MNIST can not represent modern CV tasks, as noted in this April 2017 Twitter thread, deep learning expert/Keras author François Chollet.


## Preparing the dataset

In [7]:
import numpy as np

def prepare_data():
    
    # 1. load FASHION MNIST dataset
    fashionmnist_dataset = keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = fashionmnist_dataset.load_data()
    
    # 2. map image pixel values from [0,255] to [0,1]
    x_train = x_train.astype(float)
    x_test  = x_test.astype(float)
    
    x_train = x_train * (1.0/255.0)
    x_test  = x_test  * (1.0/255.0)
    
    # 3. images are 2D, prepare input as 1D input for a MLP
    x_train = x_train.reshape(-1,28*28)
    x_test  = x_test.reshape(-1,28*28)
    
    # 4. map train data target labels to one-hot encoded vectors
    y_train_onehot = np.zeros((y_train.size, y_train.max()+1))
    y_train_onehot[np.arange(y_train.size),y_train] = 1
    
    # 5. map test data target labels to one-hot encoded vectors
    y_test_onehot = np.zeros((y_test.size, y_train.max()+1))
    y_test_onehot[np.arange(y_test.size),y_test] = 1
    
    return x_train, y_train_onehot, \
           x_test,  y_test_onehot

# get FASHION MNIST data now
x_train, y_train, x_test, y_test = prepare_data()

In [8]:
x_train.shape

(60000, 784)

In [9]:
x_test.shape

(10000, 784)

In [10]:
y_train.shape

(60000, 10)

In [11]:
y_test.shape

(10000, 10)

In [12]:
y_train[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## Perceptron

In [ ]:
from tensorflow import keras

tf.keras.backend.clear_session()

nr_inputs = 28*28

model = keras.Sequential()
model.add(keras.layers.Dense(10,
                             activation="linear",
                             input_shape=(nr_inputs,)))
model.compile(optimizer='sgd',               
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

In [ ]:
history = model.fit(x_train, y_train, epochs=PARAM_NR_TRAIN_EPOCHS, validation_split=0.1)

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

## MLP

In [ ]:
tf.keras.backend.clear_session()

nr_inputs = 28*28

model = keras.Sequential()
model.add(keras.layers.Dense(200,
                             activation="relu",
                             input_shape=(nr_inputs,)))
model.add(keras.layers.Dense(50,
                             activation="relu"))
model.add(keras.layers.Dense(10,
                             activation="linear"))
model.compile(optimizer='sgd',               
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

In [ ]:
history = model.fit(x_train, y_train, epochs=PARAM_NR_TRAIN_EPOCHS, validation_split=0.1)

In [ ]:
model_test(model, x_test, y_test, PARAM_NR_TEST_IMAGES)

## CNN

In [13]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras import losses

K.clear_session()

model = keras.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu', name="FC"))

model.add(layers.Dense(10, activation='linear', name="output"))

model.compile(loss=losses.mean_squared_error, optimizer='sgd')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 16)          4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 16)                0

In [14]:
model_test(model,
           x_test,
           y_test,
           how_many_to_test=500,
           model_type="CNN"
          )

tested so far 0 of 500 images
tested so far 100 of 500 images
tested so far 200 of 500 images
tested so far 300 of 500 images
tested so far 400 of 500 images


0.128

In [16]:
history = model.fit(x_train.reshape(60000, 28,28,1),
                    y_train,
                    epochs=int(PARAM_NR_TRAIN_EPOCHS/2),
                    validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/150
54000/54000 [==============================] - 108s 2ms/sample - loss: 0.0885 - val_loss: 0.0851
Epoch 2/150
54000/54000 [==============================] - 108s 2ms/sample - loss: 0.0810 - val_loss: 0.0768
Epoch 3/150
54000/54000 [==============================] - 109s 2ms/sample - loss: 0.0725 - val_loss: 0.0686
Epoch 4/150
54000/54000 [==============================] - 113s 2ms/sample - loss: 0.0657 - val_loss: 0.0626
Epoch 5/150
54000/54000 [==============================] - 111s 2ms/sample - loss: 0.0603 - val_loss: 0.0575
Epoch 6/150
54000/54000 [==============================] - 114s 2ms/sample - loss: 0.0559 - val_loss: 0.0535
Epoch 7/150
54000/54000 [==============================] - 111s 2ms/sample - loss: 0.0524 - val_loss: 0.0503
Epoch 8/150
54000/54000 [==============================] - 115s 2ms/sample - loss: 0.0495 - val_loss: 0.0476
Epoch 9/150
54000/54000 [==============================] - 113s 2ms/sample - lo

54000/54000 [==============================] - 83s 2ms/sample - loss: 0.0272 - val_loss: 0.0275
Epoch 76/150
54000/54000 [==============================] - 89s 2ms/sample - loss: 0.0271 - val_loss: 0.0273
Epoch 77/150
54000/54000 [==============================] - 88s 2ms/sample - loss: 0.0270 - val_loss: 0.0271
Epoch 78/150
54000/54000 [==============================] - 97s 2ms/sample - loss: 0.0269 - val_loss: 0.0270
Epoch 79/150
54000/54000 [==============================] - 79s 1ms/sample - loss: 0.0268 - val_loss: 0.0270
Epoch 80/150
54000/54000 [==============================] - 76s 1ms/sample - loss: 0.0267 - val_loss: 0.0270
Epoch 81/150
54000/54000 [==============================] - 85s 2ms/sample - loss: 0.0267 - val_loss: 0.0269
Epoch 82/150
54000/54000 [==============================] - 79s 1ms/sample - loss: 0.0266 - val_loss: 0.0271
Epoch 83/150
54000/54000 [==============================] - 82s 2ms/sample - loss: 0.0265 - val_loss: 0.0266
Epoch 84/150
54000/54000 [======

Epoch 150/150
54000/54000 [==============================] - 86s 2ms/sample - loss: 0.0224 - val_loss: 0.0231


In [18]:
model_test(model,
           x_test,
           y_test,
           PARAM_NR_TEST_IMAGES,
           model_type="CNN")

tested so far 0 of 10000 images
tested so far 100 of 10000 images
tested so far 200 of 10000 images
tested so far 300 of 10000 images
tested so far 400 of 10000 images
tested so far 500 of 10000 images
tested so far 600 of 10000 images
tested so far 700 of 10000 images
tested so far 800 of 10000 images
tested so far 900 of 10000 images
tested so far 1000 of 10000 images
tested so far 1100 of 10000 images
tested so far 1200 of 10000 images
tested so far 1300 of 10000 images
tested so far 1400 of 10000 images
tested so far 1500 of 10000 images
tested so far 1600 of 10000 images
tested so far 1700 of 10000 images
tested so far 1800 of 10000 images
tested so far 1900 of 10000 images
tested so far 2000 of 10000 images
tested so far 2100 of 10000 images
tested so far 2200 of 10000 images
tested so far 2300 of 10000 images
tested so far 2400 of 10000 images
tested so far 2500 of 10000 images
tested so far 2600 of 10000 images
tested so far 2700 of 10000 images
tested so far 2800 of 10000 imag

0.8404

# Results

| Model         | Dataset       | Acc test data  | Nr train epochs |
|---------------|---------------|----------------|-----------------|
| Perceptron    | MNIST         | 85%            | 300             |
| MLP 200-50-10 | MNIST         | 97%            | 300             |
| CNN           | MNIST         |                | 150             |
| Perceptron    | Fashion-MNIST | 81%            | 300             |
| MLP 200-50-10 | Fashion-MNIST | 88%            | 300             |
| CNN           | Fashion-MNIST | 84%            | 150             |